<a href="https://colab.research.google.com/github/gentleman101/Deep_Learning/blob/main/Airplane_recognition_MTARSI_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing necessary packages

Basic Idea will be to import a pre-trained model and then design a classifier on top of it. For our purpose I am taking inceptionV3 as our base model to identify object.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

## Importing Pre-Trained model

In [2]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model,load_model
# importing inception_v3 net from Keras with imagenet weights in conv_base
conv_base =  InceptionV3(weights='imagenet',include_top=False,input_shape=(300, 300, 3))

87924736/87910968 [==============================] - 1s 0us/step


In [3]:
output = conv_base.layers[-1].output
output = keras.layers.Flatten()(output)
model_tl = Model(conv_base.input, output)

In [4]:
# Setting all the base layers as untrainable
model_tl.trainable = False
for layer in model_tl.layers:
  layer.trainable = False

# To read whether the layers are properly set or not
layers = [(layer, layer.name, layer.trainable) for layer in  model_tl.layers]
model_layers=pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
print(model_layers) 

                                            Layer Type  ... Layer Trainable
0    <keras.engine.input_layer.InputLayer object at...  ...           False
1    <keras.layers.convolutional.Conv2D object at 0...  ...           False
2    <keras.layers.normalization_v2.BatchNormalizat...  ...           False
3    <keras.layers.core.Activation object at 0x7fcc...  ...           False
4    <keras.layers.convolutional.Conv2D object at 0...  ...           False
..                                                 ...  ...             ...
307  <keras.layers.merge.Concatenate object at 0x7f...  ...           False
308  <keras.layers.merge.Concatenate object at 0x7f...  ...           False
309  <keras.layers.core.Activation object at 0x7fcb...  ...           False
310  <keras.layers.merge.Concatenate object at 0x7f...  ...           False
311  <keras.layers.core.Flatten object at 0x7fcbfa6...  ...           False

[312 rows x 3 columns]


## Train Test Split

In [5]:
#import splitfolders

In [6]:
# Splitting the folder into 75% train and 25% test. I will form validation set using train data later
#splitfolders.ratio('/content/drive/MyDrive/DATA/MTARSI/airplane-dataset-trans', output="'/content/drive/MyDrive/DATA/MTARSI/", seed=1337, ratio=(.75, .25,), group_prefix=None) # default values

## Image Processing

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [8]:
# Describing utility parameters
batch_size=15
epochs=30
target_size=(300,300)

In [9]:
test_path='/content/drive/MyDrive/DATA/MTARSI/test_2'
train_path='/content/drive/MyDrive/DATA/MTARSI/train_2'

In [10]:
# Initiationg ImageDataGenerator on Keras to have a new augmented dataset
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3,  
rotation_range=50,
width_shift_range=0.2, 
 height_shift_range=0.2, 
shear_range=0.2,
horizontal_flip=True,
brightness_range = [0.8, 1.2],
fill_mode='nearest',        
 validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=target_size,#  
batch_size=batch_size,
class_mode='categorical',
subset='training')
validation_generator = train_datagen.flow_from_directory(
train_path,
target_size=target_size,
batch_size=batch_size,
class_mode='categorical',
subset='validation')

Found 6979 images belonging to 23 classes.
Found 1732 images belonging to 23 classes.


## Building Model Architecture

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras import optimizers

In [16]:
# building a linear stack of layers with the sequential model
model =Sequential()
model.add(model_tl)
# # hidden layer
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# # hidden layer
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# # hidden layer
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# # output layer
model.add(Dense(23, activation='softmax'))
# compiling the sequential model
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 131072)            21802784  
_________________________________________________________________
dense_2 (Dense)              (None, 23)                3014679   
Total params: 24,817,463
Trainable params: 3,014,679
Non-trainable params: 21,802,784
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Creating Checkpoints for model

In [17]:
from keras.callbacks import *
filepath="/content/drive/My Drive/MyCNN/RR/epochs:{epoch:03d}-val_acc: {val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=False,save_freq='epoch',mode='max')
callbacks_list = [checkpoint]

## Training the model

In [ ]:
history = model.fit(train_generator,steps_per_epoch=train_generator.samples//batch_size,validation_data=validation_generator,
validation_steps=validation_generator.samples//batch_size,
epochs=epochs,verbose=1,shuffle=True,callbacks=callbacks_list)

Epoch 1/30
465/465 [==============================] - 2599s 5s/step - loss: 4.2339 - acc: 0.2870 - val_loss: 4.1703 - val_acc: 0.3130

Epoch 00001: saving model to /content/drive/My Drive/MyCNN/RR/epochs:001-val_acc: 0.313.hdf5
Epoch 2/30
465/465 [==============================] - 231s 497ms/step - loss: 2.2985 - acc: 0.5360 - val_loss: 3.8986 - val_acc: 0.4574

Epoch 00002: saving model to /content/drive/My Drive/MyCNN/RR/epochs:002-val_acc: 0.457.hdf5
Epoch 3/30
465/465 [==============================] - 232s 499ms/step - loss: 2.0068 - acc: 0.6094 - val_loss: 3.9944 - val_acc: 0.4771

Epoch 00003: saving model to /content/drive/My Drive/MyCNN/RR/epochs:003-val_acc: 0.477.hdf5
Epoch 4/30
465/465 [==============================] - 232s 499ms/step - loss: 1.8948 - acc: 0.6334 - val_loss: 3.9287 - val_acc: 0.4794

Epoch 00004: saving model to /content/drive/My Drive/MyCNN/RR/epochs:004-val_acc: 0.479.hdf5
Epoch 5/30
465/465 [==============================] - 235s 507ms/step - loss: 1.85

## Model Performance on Validation Set

In [ ]:
# Model evaluation
scores_train = model.evaluate(train_generator,verbose=1)
scores_validation = model.evaluate(validation_generator,verbose=1)
print("Train Accuracy: %.2f%%" % (scores_train[1]*100))
print("Validation Accuracy: %.2f%%" % (scores_validation[1]*100))

In [ ]:
#For plotting Accuracy and Loss
def LearningCurve(history):
  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

In [ ]:
LearningCurve(history)

## Making Predictions

In [ ]:
import math
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
test_steps = compute_steps_per_epoch(100)
test_generator = test_datagen.flow_from_directory(test_path,target_size=target_size, batch_size=batch_size,class_mode=None,shuffle=False)
test_generator.reset()
#Calling the saved model for making predictions
tl_img_aug_cnn = load_model('/content/drive/My Drive/MyCNN/RR/epochs:025-val_acc: 0.592.hdf5')
pred=tl_img_aug_cnn.predict(test_generator,verbose=1,steps=test_steps)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames, "Predictions":predictions})
#create a function for visualizing model performance
import seaborn as sns
def PerformanceReports(conf_matrix,class_report,labels):
    ax= plt.subplot()
    sns.heatmap(conf_matrix, annot=True,ax=ax)
    #labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)
    plt.show()
    ax= plt.subplot()
    sns.heatmap(pd.DataFrame(class_report).iloc[:-1, :].T,  annot=True,ax=ax)
    ax.set_title('Classification Report')
    plt.show()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
labels=['A','A-10','A-26','B-1','B-2','B-29','B-52','Background','Boeing','C-5','C-17','C-21','C-130','C-135','E-3','F','F-16','F-22','KC-10','P-63','T-6','t-43','U-2']
test_labels = [fn.split('/')[0] for fn in filenames]
cm=confusion_matrix(test_labels,predictions)
print(cm)
cr=classification_report(test_labels, predictions)
class_report=classification_report(test_labels, predictions,target_names=labels,output_dict=True)
print(cr)
PerformanceReports(cm,class_report,labels)

Found 2978 images belonging to 23 classes.


InvalidArgumentError: ignored